In [54]:
import pandas as pd

X_train = pd.read_csv('./X_train.csv', parse_dates=['reportts'])
y_train = pd.read_csv('./y_train.csv', parse_dates=['reportts'])

X_test = pd.read_csv('./X_test.csv', parse_dates=['reportts'])

dataset = X_train.merge(y_train, on=['acnum', 'pos', 'reportts']).dropna(subset=['egtm'])


In [55]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

def train_model(X, y):
  assert len(X) == len(y)

  X = X.fillna(0).drop(columns=[
    'reportts', 'acnum', 'pos', 'fltdes',	'dep', 'arr'
  ])

  # обучаем на более ранних данных, тестируем на поздних
  train_i = int(len(X) * 75 / 100)
  X_train, y_train = X[0:train_i], y[0:train_i]
  X_val, y_test = X[train_i:], y[train_i:]

  model = Ridge(alpha=5)
  model.fit(X_train, y_train)

  predicted = model.predict(X_val)
  rmse = mean_squared_error(y_test, predicted, squared=False)
  mae = mean_absolute_error(y_test, predicted)

  return rmse, mae, model

In [56]:
fleet = ['VQ-BGU', 'VQ-BDU']

for acnum in fleet:
  X = dataset[dataset['acnum'] == acnum].drop(columns=['egtm'])
  y = dataset[dataset['acnum'] == acnum]['egtm']

  rmse, mae, model = train_model(X, y)

  print(f'acnum={acnum} RMSE={rmse:.3f} MAE={mae:.3f}')


acnum=VQ-BGU RMSE=7.284 MAE=5.928
acnum=VQ-BDU RMSE=3.416 MAE=2.951
